In [ ]:
## !pip install opencv-python==4.5.5.64 opencv-contrib-python==4.5.5.64 --user
import cv2 as cv
import numpy as np
import open3d as o3d

import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares

The following 3 cells implement Scipy's Least Square Bundle Adjustment

In [ ]:
def project(points, camera_params,intrinsic):
    projections = []

    for i in range(len(camera_params)):
        R = camera_params[i][:3]
        R, _ = cv.Rodrigues(R)
        t = camera_params[i][3:6]
        point = points[i]
        point = np.expand_dims(point,axis=0)
        point,_ = cv.projectPoints(point, R, t, intrinsic , distCoeffs=np.array([]))
        point = np.squeeze(np.array(point))
        projections.append(point)

    return projections

In [ ]:
def fun(params, n_cameras, n_points, camera_indices, point_indices, points_2d,intrinsic):
    camera_params = params[:n_cameras * 9].reshape((n_cameras, 9))
    points_3d = params[n_cameras * 9:].reshape((n_points, 3))
    points_proj = project(points_3d[point_indices], camera_params[camera_indices],intrinsic)
    return (points_proj - points_2d).ravel()

In [ ]:
def bundle_adjustment_sparsity(n_cameras, n_points, camera_indices, point_indices):
    m = camera_indices.size * 2
    n = n_cameras * 9 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)
    for s in range(9):
        A[2 * i, camera_indices * 9 + s] = 1
        A[2 * i + 1, camera_indices * 9 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 9 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 9 + point_indices * 3 + s] = 1

    return A

In [ ]:
#Calculate image keypoints and descriptors
def calculate_kps_descs(imgs):
    kps = []
    des = []
    sift = cv.SIFT_create()
    for i in range(len(imgs)):
        print("Keypoint Calculation: ["+str(i/len(imgs)*100)+" %]")
        kp,d = sift.detectAndCompute(imgs[i],None)
        kps.append(kp)
        des.append(d)
    print("Keypoint Calculation: [100 %]")
    return kps,des

In [ ]:
#Matches and Lowe's ratio test
def calculate_matches(kps,des):
    matches = []
    n_imgs = len(kps)
    matcher = cv.BFMatcher()
    for i in range(n_imgs):
        print("Keypoint Match: ["+str(i/n_imgs*100)+" %]")
        matches.append([])
        for j in range(n_imgs):
            match = []
            m = matcher.knnMatch(des[i], des[j], k=2)
            for k in range(len(m)):
                    if m[k][0].distance < 0.7*m[k][1].distance:
                        match.append(m[k][0])
            matches[i].append(match)
    print("Keypoint Match: [100 %]")
    return matches

In [ ]:
#Find best indexes based on the number of keypoints on each match
def best_idxs(matches):
    exts = []
    for i in range(len(matches)):
        for j in range(len(matches)):
            if(i!=j):
                exts.append(len(matches[i][j]))
            else:
                exts.append(-np.Inf)
                
    exts = np.array(exts).reshape((len(matches),len(matches)))
    
    idxs = []
    for i in range(len(matches)):
        max_arg = np.argmax(exts[i])
        if(max_arg not in idxs):
            idxs.append(max_arg)
        else:
            aux = exts[i].copy()
            while(len(aux)>0 and np.argmax(aux) in idxs):
                aux = np.delete(aux,np.argmax(aux))
            if(len(aux)>0):
                idxs.append(np.argmax(aux))
            else:
                for j in range(len(matches)):
                    if(j not in idxs):
                        idxs.append(j)
    return idxs

In [ ]:
#Reorders the dataset using the recovered best indexes
def reorder(imgs,kps,des,matches,imgs_color,idxs):
    imgs = np.array(imgs,dtype=object)[idxs].tolist()
    kps = np.array(kps,dtype=object)[idxs].tolist()
    des = np.array(des,dtype=object)[idxs].tolist()
    matches = np.array(matches,dtype=object)[idxs][:,idxs].tolist()
    imgs_color = np.array(imgs_color,dtype=object)[idxs].tolist()
    return imgs,kps,des,matches,imgs_color

In [ ]:
#Loads the dataset
def load_images(dataset_path,fill,num_imgs,start_idx):
    print("Loading Images...")
    imgs = []
    imgs_color = []
    numimgs = num_imgs
    for i in range(start_idx,numimgs+1):
        imgs.append(cv.imread(dataset_path+'{}'.format(str(i).zfill(fill)+".png"),0))
        imgs_color.append(cv.cvtColor(cv.imread(dataset_path+'{}'.format(str(i).zfill(fill)+".png")),cv.COLOR_BGR2RGB))
    print("Images Loaded.")
    return imgs,imgs_color

In [ ]:
#Recoveries the dataset path
def get_path_params(dataset):
    #Default = temple
    dense = True
    fill = 4
    dataset_path = "temple/temple"
    reorder_data = True
    intrinsic = np.array([[1520.40,0,302.32],[0,1525.90,246.87],[0,0,1]])
    num_imgs = 312
    start_idx = 1
    if(dataset == "fountain"):
        dense = False
        fill = 4
        dataset_path = "fountain/"
        reorder_data = False
        intrinsic = np.array([[2759.48,0,1520.69 ],[0,2764.16,1006.81 ],[0,0,1]])
        num_imgs = 11
        start_idx = 0
    elif(dataset=="castle"):
        dense = False
        fill = 4
        dataset_path = "castle/"
        reorder_data = False
        intrinsic = np.array([[2759.48,0,1520.69 ],[0,2764.16,1006.81 ],[0,0,1]])
        num_imgs = 10
        start_idx = 0
    elif(dataset=="dino"):
        dense = True
        fill = 4
        dataset_path = "dino/dino"
        reorder_data = True
        intrinsic = np.array([[3310.40,0,316.73],[0,3325.50,200.55],[0,0,1]])
        num_imgs = 363
        start_idx = 1
    return dataset_path,fill,dense,reorder_data,intrinsic,num_imgs+(start_idx-1),start_idx,num_imgs

In [ ]:
#Performs the functions mentioned above and returns the value required for the Reconstruct class
def pre_process(dataset):
    dataset_path,fill,dense,reorder_dataset,intrinsic,num_imgs,start_idx,def_imgs = get_path_params(dataset)
    imgs,imgs_color = load_images(dataset_path,fill,num_imgs,start_idx)
    kps,des = calculate_kps_descs(imgs)
    matches = calculate_matches(kps,des)
    if(reorder_dataset):
        print("Reordering...")
        idxs = best_idxs(matches)
        imgs,kps,des,matches,imgs_color = reorder(imgs,kps,des,matches,imgs_color,idxs)
        print("Reordering Complete.")
    return imgs,imgs_color,kps,des,matches,dense,intrinsic,def_imgs

In [ ]:
#Saves parameters for BA
class CameraParams:
    def __init__(self):
        self.cameras = np.array([])
        self.camera_indices = np.array([])
        self.point_indices = np.array([])
        self.points_2D = np.array([])
        self.points_3D = np.array([])
        
class InputError(Exception):
    def __init__(self, message):            
        super().__init__(message)

In [ ]:
class Reconstruct:
    def __init__(self,imgs_color,kps,des,matches,dense,intrinsic):
        self.matched = matches
        self.intrinsic = intrinsic
        self.points_3D = []
        self.extrinsics = []
        self.descriptors = des
        self.kps = kps
        self.masks = np.zeros(len(kps),dtype=object)
        for i in range(len(self.masks)):
            self.masks[i] = np.zeros(len(kps[i]))           
        self.matchedkps = []
        self.matcheddes = []
        self.camera = CameraParams()
        self.final_error = 0
        self.res = 0
        self.colors = []
        self.imgs_color = imgs_color
        self.do_ba = False
        self.dense = dense
        
    def median(self,x):
        m,n = x.shape
        middle = np.arange((m-1)>>1,(m>>1)+1)
        x = np.partition(x,middle,axis=0)
        return x[middle].mean(axis=0)

    def remove_outliers(self,thresh=10):           
        m = self.median(self.points_3D)                            
        s = np.abs(self.points_3D-m)                          
        self.points_3D = np.array(self.points_3D[(s<self.median(s)*thresh).all(axis=1)])
        self.colors= np.array(self.colors[(s<self.median(s)*thresh).all(axis=1)])
        self.matcheddes = self.matcheddes[(s<self.median(s)*thresh).all(axis=1)]
        
    def remove_outliers_knn(self):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector((self.points_3D))
        cl, ind = pcd.remove_statistical_outlier(nb_neighbors=5,
                                                            std_ratio=1)
        self.colors= np.array(self.colors[ind])
        self.points_3D = np.array(self.points_3D[ind])
        self.matcheddes = self.matcheddes[ind]
    
#Find matches and filter with the fundamental matrix, only non-matched points will be returned
    def getMatches(self,idx1,idx2):
        matches = self.matched[idx1][idx2]
        kp = self.kps[idx1]
        kp2 = self.kps[idx2]
        des1 = self.descriptors[idx1]
        des2 = self.descriptors[idx2]
        finalmatches = []
        for m in matches:
                finalmatches.append(m)
        match1 = np.array(np.float32([kp[m.queryIdx].pt for m in finalmatches]).reshape(-1,1,2))
        match2 = np.array(np.float32([kp2[m.trainIdx].pt for m in finalmatches]).reshape(-1,1,2))
        color = np.int32(np.floor(match1))
        color = color.reshape((color.shape[0],2))
        colors = []
        for x,y in color:
            colors.append(self.imgs_color[idx1][y][x])
        des1f = ([des1[m.queryIdx].tolist() for m in finalmatches])
        des1f = np.array(des1f,dtype=np.float32)
        des2f = ([des2[m.trainIdx].tolist() for m in finalmatches])
        des2f = np.array(des2f,dtype=np.float32)
        F, mask = cv.findFundamentalMat(match1, match2, method=cv.FM_RANSAC,
                                     ransacReprojThreshold=3, confidence=0.99)
        colors = np.array(colors)
        auxmask = mask.copy();
        if(mask is not None):
            mask = mask.flatten()
            valid = True
            count = np.count_nonzero(mask == 1)
            if(count>(0.2*len(finalmatches))):
                i=0
                for m in finalmatches:
                    aux = mask[i]
                    if(self.masks[idx1][m.queryIdx] == 1 and self.masks[idx2][m.trainIdx] == 1):
                        mask[i] = 0
                    if(self.masks[idx1][m.queryIdx] == 0):
                        self.masks[idx1][m.queryIdx] = aux
                    if(self.masks[idx2][m.trainIdx] == 0):
                        self.masks[idx2][m.trainIdx] = aux
                    i = i+1
            else:
                valid = False
        else:
            valid = False
        return match1,match2,des1f,des1f,mask.astype(bool),valid,auxmask,colors
        
#Performs Bundle Adjustment
    def BA(self):
        camera_point_indices = np.int32(np.hstack((self.camera.point_indices)))
        print("point_idx: ",camera_point_indices.shape)
        camera_points_2D = self.camera.points_2D.reshape((self.camera.points_2D.shape[0],2))
        camera_indices = np.int32(np.hstack((self.camera.camera_indices)))
        print("camera_idx: ",self.camera.camera_indices.shape)
        camera_points_3D = np.array(self.points_3D)
        print("camera_points_3D: ",camera_points_3D.shape)
        cameras = self.camera.cameras.reshape((len(self.extrinsics[0]),9))
        print("cameras: ",cameras.shape)
        x0 = np.hstack((cameras.ravel(), camera_points_3D.ravel()))
        n_cameras = cameras.shape[0]
        n_points = self.points_3D.shape[0]
        n = 9 * n_cameras + 3 * n_points
        m = 2 * camera_points_2D.shape[0]
        f0 = fun(x0, n_cameras, n_points, camera_indices, camera_point_indices, camera_points_2D,self.intrinsic)
        A = bundle_adjustment_sparsity(n_cameras, n_points, camera_indices, camera_point_indices)
        res = least_squares(fun, x0, jac_sparsity=A, verbose=0,x_scale='jac', ftol=1e-6, method='trf',
                        args=(n_cameras, n_points, camera_indices, camera_point_indices, camera_points_2D,self.intrinsic))
        self.points_3D = res.x[n_cameras * 9:].reshape(n_points, 3)
        cams = res.x[:n_cameras * 9].reshape(n_cameras, 9)
        aux_ext = []
        R,_ = cv.Rodrigues(cams[0][:3])
        t = cams[0][3:6].reshape((3,1))
        ext = np.hstack((R,t))
        R,_ = cv.Rodrigues(cams[0][:3])
        t = cams[0][3:6].reshape((3,1))
        ext2 = np.hstack((R,t))
        aux_ext.append([ext,ext2])
        for i in range(2,len(cams)):
            aux_ext[0].append(ext)
        self.extrinsics = aux_ext

#Finds the baseline 3D points from the essential matrix
    def getBaselinePoints(self,idx1,idx2):
        points3D = np.zeros((3,3))
        match1,match2,des1,des2,mask,_,_,colors = self.getMatches(idx1,idx2)       
        match1 = match1[mask]
        match2 = match2[mask]
        colors = colors[mask]
        des1 = des1[mask]
        des2 = des2[mask]
        self.matchedkps.append([match1,match2])
        ess,_ = cv.findEssentialMat(match1,match2,self.intrinsic,np.zeros((8,1)),self.intrinsic,np.zeros((8,1)),method=cv.FM_RANSAC)
        
        if (ess is None):
            return None
        ess = ess[0:3,0:3]
        _, R, t,mask_ch = cv.recoverPose(ess,match1,match2,self.intrinsic)
        mask_ch = (mask_ch[:,0].astype('bool'))
        ext = np.hstack((R,t))
        P = self.intrinsic@ext
        ess2,_ = cv.findEssentialMat(match2,match1,self.intrinsic,np.zeros((8,1)),self.intrinsic,np.zeros((8,1)),method=cv.FM_RANSAC)
        if (ess2 is None):
                return None
        ess2 = ess2[0:3,0:3]
        _, R2, t2,mask_ch2 = cv.recoverPose(ess2,match2,match1,self.intrinsic)
        ext2 = np.hstack((R2,t2))
        P2 = self.intrinsic@ext2
        self.extrinsics.append([ext,ext2])
        mask_ch2 = (mask_ch2[:,0].astype('bool'))
        des1 = des1[mask_ch]
        des2 = des2[mask_ch]
        if(self.matcheddes == []):
            self.matcheddes = des1
        else:
            self.matcheddes = np.concatenate((np.array(self.matcheddes),des1),axis=0)
            
        points_4d = cv.triangulatePoints(P, P2, match1[mask_ch],match2[mask_ch2])
        points_3D = cv.convertPointsFromHomogeneous(points_4d.transpose())
        points_3D = np.reshape(points_3D,(points_3D.shape[0],3))
        self.points_3D = points_3D
        self.colors = colors[mask_ch]
        if(self.do_ba):
            Rm,_ = cv.Rodrigues(R)
            Rm2,_ = cv.Rodrigues(R2)
            ext_BA = np.array([Rm.flatten(),t.flatten(),[0,0,0]])
            self.camera.cameras = ext_BA
            ext_BA2 = np.array([Rm2.flatten(),t2.flatten(),[0,0,0]])
            self.camera.cameras = np.vstack((self.camera.cameras,ext_BA2))
            self.camera.points_2D = np.vstack((match1[mask_ch],match2[mask_ch2]))
            self.camera.camera_indices = np.concatenate([np.ones(len(match1[mask_ch]))*idx1,np.ones(len(match2[mask_ch2]))*idx2])
            self.camera.point_indices = np.concatenate([np.arange(len(points_3D)),np.arange(len(points_3D))])

#Solves the PnP problem and continues the reconstructios
    def getPnPPoints(self,idx):
        kp = self.kps[idx]
        des = self.descriptors[idx]
        matcher = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
        matches = matcher.match(self.matcheddes,des)
        finalmatches = []
        for m in matches:
                finalmatches.append(m)
        points_2D = np.array(np.float32([kp[m.trainIdx].pt for m in finalmatches]).reshape(-1,1,2))
        points_2D = points_2D.reshape(points_2D.shape[0],2)
        points_3D = np.array(np.float32([self.points_3D[m.queryIdx] for m in finalmatches]))
        if(len(points_2D)>=4):
            _, R, t, _ = cv.solvePnPRansac(points_3D, points_2D, self.intrinsic, None,
                                                confidence=0.99, reprojectionError=8, flags=cv.FM_RANSAC)
            des1 = ([des[m.trainIdx].tolist() for m in finalmatches])
            des1 = np.array(des1,dtype=np.float32)
            R2 = R.copy()
            R,_ = cv.Rodrigues(R)
            ext = np.hstack((R,t))
            P = self.intrinsic@ext
            for i in range(len(self.extrinsics[0])):
                if(i!=idx):
                    if(np.linalg.norm(ext-self.extrinsics[0][i])<=2 and (len(self.matched[i][idx])>100)):
                        P2 = self.intrinsic@self.extrinsics[0][i]
                        points_2D = points_2D.reshape(points_2D.shape[0],1,2)
                        ax1,ax2,desax1,desax2,mask,valid,full_mask,colors = self.getMatches(i,idx)
                        if(valid and mask.any()):
                            mask = mask.astype(bool).flatten()
                            full_mask = full_mask.astype(bool).flatten()
                            points_4d = cv.triangulatePoints(P, P2,ax2[mask],ax1[mask])
                            points_3D = cv.convertPointsFromHomogeneous(points_4d.transpose())
                            points_3D = np.reshape(points_3D,(points_3D.shape[0],3))
                            self.points_3D = np.reshape(self.points_3D,(self.points_3D.shape[0],3))
                            self.points_3D = np.concatenate((self.points_3D,points_3D),axis=0)
                            self.colors = np.concatenate((self.colors,colors[mask]),axis=0)
                            self.matcheddes = np.concatenate((np.array(self.matcheddes),desax1[mask]),axis=0)
                            if(self.do_ba):
                                points_4d_BA = cv.triangulatePoints(P, P2,ax2[full_mask],ax1[full_mask])
                                points_3D_BA = cv.convertPointsFromHomogeneous(points_4d_BA.transpose())
                                points_3D_BA = np.reshape(points_3D_BA,(points_3D_BA.shape[0],3))
                                indices = np.ones((len(points_3D_BA)*2))*-1
                                for l in range(len(points_3D_BA)):
                                    for k in range(len(self.points_3D)):
                                        if((self.points_3D[k]==points_3D_BA[l]).all()):
                                            indices[l] = k
                                            indices[l+len(points_3D_BA)] = k
                                            break
                                self.camera.points_2D = np.vstack((self.camera.points_2D,ax1[full_mask]))
                                self.camera.points_2D = np.vstack((self.camera.points_2D,ax2[full_mask]))
                                self.camera.camera_indices = np.concatenate([self.camera.camera_indices,np.ones(len(ax1[full_mask]))*i])
                                self.camera.camera_indices = np.concatenate([self.camera.camera_indices,np.ones(len(ax2[full_mask]))*idx])
                                self.camera.point_indices = np.concatenate([self.camera.point_indices,indices])
            if(self.do_ba):
                ext_BA2 = np.array([R2.flatten(),t.flatten(),[self.intrinsic[0][0],0,0]])
                self.camera.cameras = np.vstack((self.camera.cameras,ext_BA2))
            self.extrinsics[0].append(ext)

    def perform_reconstruction(self,n_imgs,do_ba=False,ba_limit = 30):
        if(n_imgs>len(self.imgs_color)):
            raise InputError("Number of images must be smaller than or equal the number of loaded images.")
        self.do_ba = do_ba
        self.getBaselinePoints(0,1)
        for i in range(2,n_imgs):
            print("Point Cloud Reconstruction: ["+str(i/n_imgs*100)+" %]")
            r.getPnPPoints(i)
            if(not do_ba and i%10 == 0):
                if(len(r.points_3D)>0):
                    r.remove_outliers(5)
            if(i == ba_limit-1 and do_ba):
                self.BA()
                self.do_ba=False
        r.remove_outliers_knn()
        print("Point Cloud Reconstruction: [100 %]")
        print("Number of points: "+str(len(self.points_3D)))

In [ ]:
dataset,num_scenes = "temple",312
#dataset,num_scenes = "fountain",11
#dataset,num_scenes = "castle",10
#dataset,num_scenes = "dino",363; Not enough matches

In [ ]:
#loads the dataset and does keypoint finding and matching
_,imgs_color,kps,des,matches,dense,intrinsic,def_imgs = pre_process(dataset)

In [ ]:
#Performs the Reconstruction
r = Reconstruct(imgs_color,kps,des,matches,dense,intrinsic)
r.perform_reconstruction(num_scenes)

In [ ]:
#Shows the point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector((r.points_3D))
pcd.colors = o3d.utility.Vector3dVector((r.colors/255.0))
o3d.visualization.draw_geometries([pcd])

In [ ]:
#Alpha shape
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector((r.points_3D))
pcd.colors = o3d.utility.Vector3dVector((r.colors/255.0))
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=15)
alpha = 0.04
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
#Ball Pivoting
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector((r.points_3D))
pcd.colors = o3d.utility.Vector3dVector((r.colors/255.0))
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=15)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
factor = 2
radius = factor * avg_dist   
radi =  o3d.utility.DoubleVector([radius, radius * 2])
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radi)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
#Poisson Reconstruction
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector((r.points_3D))
pcd.colors = o3d.utility.Vector3dVector((r.colors/255.0))
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=15)
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
o3d.visualization.draw_geometries([mesh])

## 